In [153]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats

N_DATA = 2000
N_BINS = 60
data = np.random.normal(4, 3, N_DATA)
data = np.where(data < 0, (-1) * abs(data) ** 2.1, abs(data) ** 2.8)

In [154]:
# data에 대해 Gaussian PDF의 y값을 생성하여 x값과 함께 반환

# args
# - data : NumPy array 형태로 주어진 데이터

def get_gaussian(data):
    mean = np.mean(data)
    std = np.std(data)

    x = np.linspace(np.min(data), np.max(data), 10000)
    pdf = stats.norm.pdf(x, loc=mean, scale=std) # Gaussian PDF
    return x, pdf

In [155]:
# 데이터를 시각화하여 Gaussian graph와 함께 표시

# args
# - data : NumPy array 형태로 주어진 데이터

def visualize_data_with_gaussian(data, chart_title=''):
    x, pdf = get_gaussian(data)
    n_data = len(data)

    fig = go.Figure()

    # 일정 개수의 bin을 갖는 histogram을 그리기 위해서는 아래처럼 코드를 작성해야 함
    histogram_bin_size = (np.max(data) - np.min(data)) / N_BINS
    pdf_for_visual = histogram_bin_size * n_data * pdf

    fig.add_trace(go.Histogram(x=data,
                              xbins=dict(start=np.min(data),
                                          end=np.max(data),
                                          size=histogram_bin_size),
                              opacity=1.0,
                              name='histogram'))

    fig.add_trace(go.Scatter(x=x, y=pdf_for_visual,
                            mode='lines',
                            fill='tozeroy',
                            line_color='green',
                            name='Gaussian'))

    fig.update_layout(title_text=chart_title)
    fig.update_layout(width=700, height=400)
    fig.update_layout(bargap=0.2)
    fig.show()

In [156]:
# 초기 데이터
visualize_data_with_gaussian(data, chart_title='0. original data distribution')

In [157]:
# 1. y = log2 (x + 1) 로 normalize 시도
def log2p1(x):
    return np.log2(x + 1.0)

data_log2p1 = np.vectorize(log2p1)(data)

visualize_data_with_gaussian(data_log2p1, chart_title='1. y=log2(x + 1)')

<ipython-input-157-ec4c96989957>:3: RuntimeWarning:

invalid value encountered in log2



In [158]:
# 2. 최솟값의 절댓값을 먼저 더해서 모든 값을 0 이상으로 만든 후, y = log2 (x + 1) 로 normalize 시도
data_pabsmin = data - np.min(data)

visualize_data_with_gaussian(data_pabsmin, chart_title='2. added abs(min)')

In [159]:
data_pabsmin_log2p1 = np.vectorize(log2p1)(data_pabsmin)

visualize_data_with_gaussian(data_pabsmin_log2p1, chart_title='2. added abs(min) -> y = log2(x + 1)')

In [160]:
data_pabsmin_log2p1_norm = stats.zscore(data_pabsmin_log2p1)

visualize_data_with_gaussian(data_pabsmin_log2p1_norm, chart_title='2. added abs(min) -> y = log2(x + 1) -> normalize')

In [161]:
# 3. 양수 -> 제곱근, 음수 -> 절댓값의 제곱근 * (-1) 을 적용하여 normalize 시도

def new_sqrt(x):
    if x >= 0:
        return np.sqrt(x)
    else:
        return (-1.0) * np.sqrt(abs(x))

data_sqrt = np.vectorize(new_sqrt)(data)

visualize_data_with_gaussian(data_sqrt, chart_title='3. sqrt')

In [162]:
data_sqrt_norm = stats.zscore(data_sqrt)

visualize_data_with_gaussian(data_sqrt_norm, chart_title='3. sqrt -> normalize')

In [163]:
# 3a. 위 결과는 왼쪽으로 약간 치우쳐 있음 -> 이것을 보다 정교하게 normalize 하기 위해, sqrt 된 데이터에 그 최솟값을 더한 후 y=log2(x+1) 를 사용
data_sqrt_pabsmin = data_sqrt - np.min(data_sqrt)

visualize_data_with_gaussian(data_sqrt_pabsmin, chart_title='3a. sqrt -> add abs(min)')

In [164]:
data_sqrt_pabsmin_log2p1 = np.vectorize(log2p1)(data_sqrt_pabsmin)

visualize_data_with_gaussian(data_sqrt_pabsmin_log2p1, chart_title='3a. sqrt -> add abs(min) -> y = log2(x + 1)')

In [165]:
data_sqrt_pabsmin_log2p1_norm = stats.zscore(data_sqrt_pabsmin_log2p1)

visualize_data_with_gaussian(data_sqrt_pabsmin_log2p1_norm, chart_title='3a. sqrt -> add abs(min) -> y = log2(x + 1) -> normalize')

In [166]:
# 이후 outlier를 제거하는 등 추가 처리하면 됨